# 🤖 Agent-Based RAG con LangChain

## 🧠 Cos’è un Agente?

Un **agente** in AI è un'entità in grado di:

* Prendere decisioni
* Eseguire azioni
* Raggiungere obiettivi specifici

In LangChain, un **LLM agisce come il cervello dell’agente** e può usare **strumenti esterni** per ottenere risposte migliori o svolgere compiti.

---

## 🔧 Strumenti che un agente può usare

* Database vettoriali (es. Chroma, Qdrant)
* API esterne
* Ricerca Web
* Calcolatori, strumenti personalizzati, ecc.

---

## 🗂️ Prompt REACT: Reasoning + Action

LangChain supporta l’agente **REACT**, che lavora in due fasi:

1. **Osservazione** del contesto (Observation)
2. **Azione** appropriata (Action)

LangChain Hub fornisce un prompt REACT preconfigurato:

```python
from langchain import hub

prompt = hub.pull("hwchase17/react")
```

📌 Questo prompt gestisce:

* Messaggi di sistema
* Input dell’utente
* Output modellati

---

## ⚙️ Setup completo di un agente con LangChain

### 1. Caricamento Documenti + Retriever

In [23]:
from dotenv import load_dotenv
load_dotenv()

from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")

prompt.messages

c:\Users\felip\Desktop\import-pc\Advanced_RAG\.venv\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [24]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores.chroma import Chroma
from langchain_openai import OpenAIEmbeddings

loader = DirectoryLoader("./data", glob="**/*.txt")

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=120, 
    chunk_overlap=20,
    length_function=len, 
    is_separator_regex=False
)

chunks = text_splitter.split_documents(docs)

embedding_function = OpenAIEmbeddings()

db = Chroma.from_documents(docs, embedding_function)

retriever = db.as_retriever()

---

### 2. Creazione del Tool dal Retriever


In [25]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever=retriever, name="ragagent", description="performs RAG on a small dataset"
)

tools = [tool]



---

### 3. Setup Agente + AgentExecutor


In [26]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

`AgentExecutor` è il runtime di un agente. è questo che chiama l'agente.

Anche l'`AgentExecutor` implementa l'interfaccia Runnable, quindi usa metodi come .invoke() e .stream()

In [27]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)


agent_executor = AgentExecutor(agent=agent, tools=tools)

---

### 4. Esecuzione con `invoke()`

In [28]:
agent_executor.invoke({"input": "Who is the owner of the restaurant?"})

{'input': 'Who is the owner of the restaurant?',
 'output': 'Could you please specify the name of the restaurant you are inquiring about?'}

---

## 🔗 Note tecniche

* `AgentExecutor` è il **runtime** che gestisce l’intero flusso: richiama il prompt, interroga i tools, interpreta i risultati.
* Funziona come un `Runnable`, quindi supporta `.invoke()` e `.stream()`
* Il prompt REACT è molto utile perché guida l'agente a **pensare prima di agire**

---

## 🚀 Approfondimento: LangGraph

LangChain consiglia LangGraph se:

* Vuoi **workflow agenziali complessi**
* Hai **flussi condizionali, branching, parallelizzazione**
* Vuoi orchestrare **task LLM e tool** in modo modulare e controllato

📚 *LangGraph = libreria costruita sopra LangChain per creare flussi personalizzati con logica più esplicita*

---

## 📌 Conclusione

| Componente      | Funzione                                                  |
| --------------- | --------------------------------------------------------- |
| `LLM`           | Il cervello dell’agente                                   |
| `Tool`          | Funzione esterna utilizzabile (retriever, API, ecc.)      |
| `Prompt REACT`  | Struttura logica per pensare e agire                      |
| `AgentExecutor` | Esegue il ciclo completo osservazione → azione con i tool |

---

🎯 **Riepilogo**: Un agente RAG utilizza un **LLM + Retriever come tool**, orchestrati da un `AgentExecutor` con prompt REACT, per effettuare recupero e ragionamento più potenti.
